In [2]:
import torch

In [3]:
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 2.8.0.dev20250412+cu128


In [4]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("cuDNN version:", torch.backends.cudnn.version())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

PyTorch version: 2.8.0.dev20250412+cu128
CUDA available: True
cuDNN version: 90701
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU


In [5]:
# Import pipeline, AutoTokenizer, and AutoModelForCausalLM from transformers
# pipeline : this is a high-level API for using models easily, it gives you a simple interface to use models for various tasks. example: text generation, translation, etc.
# AutoTokenizer : this is used to convert text into tokens that the model can understand. It handles the preprocessing of text data.
# AutoModelForCausalLM : this is a class for loading pre-trained models for causal language modeling tasks. It allows you to use models like GPT-2, GPT-3, etc.

# Import the BitsAndBytesConfig class for quantization. 
# BitsAndBytesConfig : this is used for configuring quantization settings for models. It helps in reducing the model size and improving inference speed without significant loss in performance.
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [6]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
# # Model name from Hugging Face
# # model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "mistralai/Mistral-7B-Instruct-v0.3"

# # Configure 4-bit quantization
# quant_config = BitsAndBytesConfig(load_in_4bit=True)

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# # Load model with quantization and automatic device placement
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=quant_config,
#     device_map="auto" # Automatically place model on available devices (CPU/GPU)
# )


# **What the progress bar shows**

You're running:

```python
model = AutoModelForCausalLM.from_pretrained(...)
```

This line tells Hugging Face to:

1. **Download the model weights and config** from the Hub.
2. **Quantize the model (4-bit in your case)** using `BitsAndBytesConfig`.
3. **Place it on the correct device** (`device_map="auto"` handles that).

---

### The progress bars:

#### `config.json`  
- This file defines the model architecture and tokenizer setup.

---

#### ⚙️ `model.safetensors.index.json`  
- This tells Hugging Face how to **split and reference** the model weights.
- Since these big models are often too large for a single file, they’re split into chunks (`model-00001`, `00002`, etc.).

---

#### 📦 The 3 large files:
- `model-00001-of-00003.safetensors`  
- `model-00002-of-00003.safetensors`  
- `model-00003-of-00003.safetensors`

Each of these is a **partial weight file** (chunks of the full model parameters):
- You're downloading ~15 GB total (around 5 GB each).
- The download is in progress, shows speed and ETA for each file.

---

### The warning:

```txt
UserWarning: 'huggingface_hub' cannot create symlinks...
```

It means:
- On Windows, creating "symlinks" (shortcut-like references) is restricted unless you're in **Developer Mode** or running Python as an **Administrator**.
- Hugging Face uses symlinks sometimes to save disk space.

---

### In Summary

You're successfully:
- Loading the Mistral 7B Instruct model
- With 4-bit quantization
- While downloading its 3 weight shards

And you will be able to run inference!

In [11]:
# Example usage of the model. This is inference, where we provide input text to the model and get a generated response.
input_text = "Explain the theory of relativity."
# Tokenize the input text and move it to the same device as the model
# The tokenizer converts the input text into a format that the model can understand (tokens).
# The return_tensors="pt" argument specifies that the output should be in PyTorch tensor format.

# The model is then used to generate text based on the input tokens.
# The max_new_tokens argument specifies the maximum number of tokens to generate in the output.

# Finally, the generated tokens are decoded back into text format using the tokenizer's decode method.
# The skip_special_tokens=True argument ensures that special tokens (like padding or end-of-sequence tokens) are not included in the final output.
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Explain the theory of relativity.

The theory of relativity, proposed by Albert Einstein in 1905 and 1915, is a fundamental theory in physics that describes the behavior of objects and energy in the universe. It consists of two parts: special relativity and general relativity.

1. Special Relativity: This theory describes the relationship between space and time at constant velocities, where the velocity of the observer is negligible compared to the velocity of light. The key principles of special relativity are:

   a. The Principle of Relativity: The laws of physics are the same for all observers in uniform relative motion.

   b. The Speed of Light is Constant: The speed of light in a vacuum is the same for all observers, regardless of their motion or the motion of the source of light.

   c. Time Dilation and Length Contraction: As an object approaches the speed of light, time appears to slow down for observers on Earth (time dilation), and the length of the object contracts (length

#### Command to view for real time GPU utilization in Command Prompt `nvidia-smi -l 1`